# Entrenamiento de un KNN

En esta hoja de trabajo veremos como elegir los mejores hiperparámetros para entrenar a un knn, así como entrenar al propio knn y ver si los hiperparámetros elegidos dan un buen rendimiento.

Empezaremos importando las librerias y funciones que utilizaremos más adelante.

In [22]:
import pandas as pd
import csv
import numpy as np
np.random.seed(357823)
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import confusion_matrix, recall_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score


# Lectura de datos y creación de los atributos y el objetivo

Vamos a empezar por importar el dataset con todos lo atributos que calculamos para la realización de este trabajo. Luego separamos el dataset en atributos por los cuales el knn se va a entrenar, estos a su vez lo separamos en discretos y continuos para tratarlos mas adelante. Y por último elegiremos el objetivo, en este caso es un objetivo categórico.

In [23]:
# Lectura del csv para la tabla con todas las métricas
nodes = pd.read_csv("../tablas/tableWithAllAtributes.csv")
nodes.head()

,id_node,name,ml_target,degree_centrality,closeness_centrality,betweenness_centrality,clustering_coefficient,Square clustering,triangles,greedy_modularity_communities,Core number,asyn_lpa_communities
0,0,Eiryyy,0.0,0.000027,0.275005,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,1,shawflying,0.0,0.000212,0.294956,1.149733e-06,0.178571,0.072344,0.000062,0.002227,0.151515,0.0
2,2,JpMCarrilho,1.0,0.000027,0.261845,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
3,3,SuhwanCha,0.0,0.000133,0.278718,5.316292e-05,0.000000,0.019178,0.000000,0.004454,0.090909,0.0
4,4,sunilangadi2,1.0,0.000053,0.243084,6.134318e-09,0.000000,0.000000,0.000000,0.011136,0.030303,0.0


In [24]:
# Preparación de los datos para la table con todos los atributos
atributos_discretos = ['name']
atributos_continuos = ['degree_centrality','closeness_centrality','betweenness_centrality','clustering_coefficient','Square clustering','triangles','greedy_modularity_communities','Core number','asyn_lpa_communities']
atributos = nodes.loc[:,['id_node'] + atributos_discretos + atributos_continuos]
atributos.head()

,id_node,name,degree_centrality,closeness_centrality,betweenness_centrality,clustering_coefficient,Square clustering,triangles,greedy_modularity_communities,Core number,asyn_lpa_communities
0,0,Eiryyy,0.000027,0.275005,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,1,shawflying,0.000212,0.294956,1.149733e-06,0.178571,0.072344,0.000062,0.002227,0.151515,0.0
2,2,JpMCarrilho,0.000027,0.261845,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
3,3,SuhwanCha,0.000133,0.278718,5.316292e-05,0.000000,0.019178,0.000000,0.004454,0.090909,0.0
4,4,sunilangadi2,0.000053,0.243084,6.134318e-09,0.000000,0.000000,0.000000,0.011136,0.030303,0.0


In [25]:
# Elección del objetivo
objetivo = nodes['ml_target']
objetivo.head() 

0    0.0
1    0.0
2    1.0
3    0.0
4    1.0
Name: ml_target, dtype: float64

# Tratamiento de los datos

Vamos a tener que discretizar la columna name, ya que el árbol con cadenas no puede trabajar. Pués con la siguente función convertimos las cadenas en número y a esos numero los vamos a nomrmalizar entre 0 y 1, de forma que cogeremos el máximo y el mínimo para ello.

In [26]:
# Codificación
codificador_atributos_discretos = OrdinalEncoder() # Crear una instancia de la clase correspondiente
codificador_atributos_discretos.fit(atributos[atributos_discretos]) # Usar el método fit para ajustar a los datos los parámetros de la codificación

# Vemos información sobre los atributos discretos
print("///Información sobre los atributos discretos///")
print('Número de atributos detectados:',
      f'{codificador_atributos_discretos.n_features_in_}')
print()
print('Nombres de los atributos detectados:')
print(f'{codificador_atributos_discretos.feature_names_in_}')
print()
print('Categorías detectadas de cada atributo:')
for atributo, categorías in zip(
    codificador_atributos_discretos.feature_names_in_,
    codificador_atributos_discretos.categories_):
    print(f'{atributo}: {categorías}')

# Ahora aplicamos el método transform para codificar los datos
atributos[atributos_discretos] = codificador_atributos_discretos.transform(
    atributos[atributos_discretos]
)

# Normalizamos el name
normalizador = MinMaxScaler(
    # Cada atributo se normaliza al intervalo [0, 1]
    feature_range=(0, 1)
)

# Aplicamos la normalización solo a la columna 'name'
atributos['name'] = normalizador.fit_transform(atributos[['name']])
atributos.head()

///Información sobre los atributos discretos///
Número de atributos detectados: 1

Nombres de los atributos detectados:
['name']

Categorías detectadas de cada atributo:
name: ['007arunwilson' '007jedgar' '00Kai0' ... 'timothykimemia' 'timoxley'
 'timqian']


,id_node,name,degree_centrality,closeness_centrality,betweenness_centrality,clustering_coefficient,Square clustering,triangles,greedy_modularity_communities,Core number,asyn_lpa_communities
0,0,0.061673,0.000027,0.275005,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,1,0.929866,0.000212,0.294956,1.149733e-06,0.178571,0.072344,0.000062,0.002227,0.151515,0.0
2,2,0.106687,0.000027,0.261845,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
3,3,0.191517,0.000133,0.278718,5.316292e-05,0.000000,0.019178,0.000000,0.004454,0.090909,0.0
4,4,0.969442,0.000053,0.243084,6.134318e-09,0.000000,0.000000,0.000000,0.011136,0.030303,0.0


# Elección de hiperparámetros

Vamos a elegir los mejores hiperpaámetros para entrenar nuestro knn, para ello vamos a utilizar la busqueda en rejilla, donde va hacer combinación entre que los veciones sean 1, 3, 5, 7 o  9 y que la distancia entre ellos se midan o en manhattan o euclidean.

Como verás más adelante nos da que el knn debería tener un vecino y la distancia debe ser manhattan.

In [27]:
tubería_kNN = Pipeline([
                        ('kNN', KNeighborsClassifier())])
rejilla_de_parámetros = {
    # Número de vecinos impar (tarea de clasificación binaria)
    'kNN__n_neighbors': range(1, 10, 2),
    # Considerar las distancias Manhattan y euclídea
    'kNN__metric': ['manhattan', 'euclidean']
}

In [28]:
búsqueda_en_rejilla = GridSearchCV(tubería_kNN,
                                   rejilla_de_parámetros,
                                   scoring='recall',
                                   cv=10)
búsqueda_en_rejilla.fit(atributos, objetivo)

GridSearchCV(cv=10, estimator=Pipeline(steps=[('kNN', KNeighborsClassifier())]),
             param_grid={'kNN__metric': ['manhattan', 'euclidean'],
                         'kNN__n_neighbors': range(1, 10, 2)},
             scoring='recall')

In [29]:
búsqueda_en_rejilla.best_params_

{'kNN__metric': 'manhattan', 'kNN__n_neighbors': 1}

In [30]:
búsqueda_en_rejilla.best_score_

0.5438398357289527

# Validación por retención

Ahora vamos a comprobar si los resultados de la busqueda en rejilla son acertados, para ello vamos a dividir el dataset en datos de entrenamiento y datos de prubebas, en este caso vamos a devidir en un 80% de los datos para entrenar y un 20% para comprobar como se ha entrenado.

Para ello vamo a entrenar varios knn, cambiando los hiperparámetros y ver cual nos da un mejor rendimiento y luego analizaremos que significan esos rendimientos.

In [31]:
# Dividimos los conjuntos de prueba y entrenamiento
(atributos_entrenamiento, atributos_prueba,
 objetivo_entrenamiento, objetivo_prueba) = train_test_split(
        # Conjuntos de datos a dividir, usando los mismos índices para ambos
        atributos, objetivo,
        # Tamaño del conjunto de prueba (30 % en este caso)
        test_size=.2, # El más óptimo es con el 30% de conjunto de prueba
        # Estratificación según la distribución de clases en el atributo objetivo
        stratify=objetivo,
        random_state=357823)

In [32]:
# knn un vecino y distancia manhattan
clasificador_kNN_division1 = KNeighborsClassifier(
    # Para cada ejemplo se consideran los 1 ejemplos más cercanos
    n_neighbors=1,
    # La cercanía viene determinada por la distancia euclídea
    metric='manhattan'
)
clasificador_kNN_division1.fit(atributos_entrenamiento, objetivo_entrenamiento)

KNeighborsClassifier(metric='manhattan', n_neighbors=1)

In [33]:
predicciones = clasificador_kNN_division1.predict(atributos_prueba)
predicciones

array([0., 0., 0., ..., 0., 1., 0.])

In [34]:
cm = confusion_matrix(predicciones, objetivo_prueba)
tabla_cm = pd.DataFrame(cm, index=["Positivo", "Negativo"], columns=["Positivo", "Negativo"])
print(tabla_cm)

          Positivo  Negativo
Positivo      4191      1421
Negativo      1401       527


In [35]:
recall_score(objetivo_prueba, predicciones)

0.2705338809034908

In [36]:
# knn tres vecino y distancia manhattan
clasificador_kNN_division1 = KNeighborsClassifier(
    # Para cada ejemplo se consideran los 1 ejemplos más cercanos
    n_neighbors=3,
    # La cercanía viene determinada por la distancia euclídea
    metric='manhattan'
)
clasificador_kNN_division1.fit(atributos_entrenamiento, objetivo_entrenamiento)

KNeighborsClassifier(metric='manhattan', n_neighbors=3)

In [37]:
predicciones = clasificador_kNN_division1.predict(atributos_prueba)
predicciones

array([0., 0., 0., ..., 0., 0., 0.])

In [38]:
cm = confusion_matrix(predicciones, objetivo_prueba)
tabla_cm = pd.DataFrame(cm, index=["Positivo", "Negativo"], columns=["Positivo", "Negativo"])
print(tabla_cm)

          Positivo  Negativo
Positivo      4656      1593
Negativo       936       355


In [39]:
recall_score(objetivo_prueba, predicciones)

0.1822381930184805

In [40]:
# knn un vecino y distancia euclidea
clasificador_kNN_division1 = KNeighborsClassifier(
    # Para cada ejemplo se consideran los 1 ejemplos más cercanos
    n_neighbors=1,
    # La cercanía viene determinada por la distancia euclídea
    metric='euclidean'
)
clasificador_kNN_division1.fit(atributos_entrenamiento, objetivo_entrenamiento)

KNeighborsClassifier(metric='euclidean', n_neighbors=1)

In [41]:
predicciones = clasificador_kNN_division1.predict(atributos_prueba)
predicciones

array([0., 0., 0., ..., 0., 1., 0.])

In [42]:
cm = confusion_matrix(predicciones, objetivo_prueba)
tabla_cm = pd.DataFrame(cm, index=["Positivo", "Negativo"], columns=["Positivo", "Negativo"])
print(tabla_cm)

          Positivo  Negativo
Positivo      4207      1430
Negativo      1385       518


In [43]:
recall_score(objetivo_prueba, predicciones)

0.26591375770020537

# Análisis del resultado

Como vemos el mejor score nos lo da cuando ponemos un vecino y la distancia que se calcule como manhattan. Nos da una confianza del 0.2705338809034908, como en nuestros datos hay un 75% de desarrolladores web y un 25% de desarrolladores IA, sabiendo esto y mirando la matriz de confunsión vemos que el árbol está entrenado similar a una máquina aleatoria y esto se debe a que pone demasiado casos positivos, al hacer la división en train/test, al haber más casos de desarrolladores web, habrá aprendido que la mayoria son de ese tipo y por esos hay tantos positivos, casi no pone negativo.